In [ ]:
"""
Hello y'all see this?
"""
from google.colab import drive
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Connect Colab to Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Unzip training data, run once
#!unzip drive/MyDrive/data/train.zip -d drive/MyDrive/data/

In [ ]:
# Unzip validation data, run once
#!unzip drive/MyDrive/data/validation.zip -d drive/MyDrive/data/

In [ ]:
# This code chunk puts all image paths and their labels in a dataframe
train_paths = []
train_labels = []
validation_paths = []
validation_labels = []
troot = 'drive/MyDrive/data/train-cropped'
vroot = 'drive/MyDrive/data/validation-cropped'

for f in os.listdir(troot):
  tpath = os.path.join(troot, f)
  tlabel = f[-10:-8]
  train_paths.append(tpath)
  train_labels.append(tlabel)

for f in os.listdir(vroot):
  vpath = os.path.join(vroot, f)
  vlabel = f[-10:-8]
  validation_paths.append(vpath)
  validation_labels.append(vlabel)

train_images = pd.DataFrame({'path': train_paths,
                             'label': train_labels})
validation_images = pd.DataFrame({'path': validation_paths,
                                  'label': validation_labels})

In [ ]:
def get_subset(n, data, seed=123):
  random.seed(seed)
  sub = random.sample(range(data.shape[0] + 1), n)
  return data.iloc[sub,:]

train_subset = get_subset(1000, train_images)
validation_subset = get_subset(1000, validation_images)

In [ ]:
imageWidth, imageHeight = 128, 128
imageSize = (imageWidth, imageHeight)
channel = 1

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255
).flow_from_dataframe(
    dataframe = train_subset,
    directory = None,
    x_col='path',
    y_col='label',
    class_mode='categorical',  # target is 2-D array of one-hot encoded labels
    target_size=imageSize,
    shuffle=True
    )

validation_generator = ImageDataGenerator(
    rescale=1./255
).flow_from_dataframe(
    dataframe = validation_subset,
    directory = None,
    x_col='path',
    class_mode= None,  # target is 2-D array of one-hot encoded labels
    target_size=imageSize,
    shuffle=False
    )

In [ ]:
nCategories = 5
# sequential (not recursive) model (one input, one output)
model=Sequential()
model.add(Dense(512, activation="relu",
                input_shape=(imageWidth, imageHeight, channel)))
model.add(Dense(256, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(nCategories, activation="softmax"))

history = model.fit(
    train_generator,
    epochs=1
)
phat = model.predict(validation_generator)

#dfTest['category'] = np.argmax(phat, axis=-1)
#label_map = {0:"cat", 1:"dog"}
#dfTest['category'] = dfTest['category'].replace(label_map)